# Setup

In [ ]:
# Extract and build the benchmark
!make -C io

!mkdir -p /data/lab1
!io/io-static -c lab1/iofile

In [ ]:
def strs_to_tup(strs):
    if len(strs) == 1:
        return (strs[0].strip(),None)
    else:
        return (strs[0].strip(),strs[1].strip())

def parse(cmd_out):
    tups = map(strs_to_tup, [string.split(":") for string in cmd_out])
    return dict(tups)

# Reads

## Constant IO/Varying Buffer (with cache, static/dynamic)

* Total size: 16MB (default)
* Buffer: varies from 512 to 512 * 2^16
* (io.c modified to output kbytes/sec in same format as other params)
* Run multiple times then average
* Prevent warmup effect

In [ ]:
# Imports
import time

# Setup
BUF_START = 512
BUFFER_SIZES = [512 * 2 ** exp for exp in range(0, 16)]
FILES = ["lab1/iofile", "/dev/zero"]
EXECUTABLES = ["io/io-static", "io/io-dynamic"]
IO_SIZE = BUFFER_SIZES[-1] # Keep constant at default 16MB (last value of BUFFER_SIZES)
RUNS = range(1, 16) # 15 runs

# The meat

outputs = {}
for exe in EXECUTABLES:
    print("\t ==> Running {}".format(exe))
    runs = []
    for filename in FILES:
        print("\t\t ==> Reading from {}".format(filename))
        for buffer_size in BUFFER_SIZES:
            results = []
            for i in RUNS:
                out = !{exe} -v -r -b {str(buffer_size)} -t {str(IO_SIZE)} {filename}
                parsed = parse(out)
                results.append(parsed)
                time.sleep(2) # Sleep for 2 seconds to prevent warmup effect

            runs.append(results)
            times = [float(item['time']) for item in results]
            speeds = [float(item['speed'].split(' ')[0]) for item in results] # Need to split by ' ' because output speed has units attached to it (see io.c)
            avg_time = sum(times) / len(RUNS)
            avg_speed = sum(speeds) / len(RUNS)
            buf_sz = int(results[0]['buffersize'])
            tot_sz = int(results[0]['totalsize'])
            print("\t\t\t{0:2} bytes {1:2} bytes ({2:.2f} KBytes/sec): {3:.6f}s".format(buf_sz, tot_sz, avg_speed, avg_time))
    outputs.update({exe:runs})

### DTrace


In [12]:
# D Language scripts
io_syscall_script = """
syscall::clock_gettime:return
/execname == "io-static"/
{
   self->in_benchmark = 1;
   trace(vtimestamp - self->rstart);
}

syscall::clock_gettime:entry
/execname == "io-static"/
{
   self->rstart = vtimestamp;
}
"""

TOTAL_SIZE = 16*1024*1024
BUFFER_SIZE = 512
values = dict()

# Callback invoked to process the aggregation
def simple_out(a, b, c, d):
    key = c[0]
    count = d
    if key in values:
        values[key] += d
    else:
        values[key] = d

# Now run the benchmark again but without the predicate

# Create a seperate thread to run the DTrace instrumentation
dtrace_thread = DTraceConsumerThread(io_syscall_script,
                                    chew_func=None,
                                    chewrec_func=None,
                                    walk_func=simple_out,
                                    sleep=1)

# Start the DTrace instrumentation
dtrace_thread.start()

# Run the io-static benchmark    
output_predicate = !io/io-static -r -b {str(BUFFER_SIZE)} -t {str(TOTAL_SIZE)} iofile
        
# The benchmark has completed - stop the DTrace instrumentation
dtrace_thread.stop()
dtrace_thread.join()

# Print the performance of both runs
print("With predicate {} KiBytes/sec".format(output_predicate[0].split(' ')[0]))

# Display footer to indicate that the benchmarking has finished
print_footer(["Finished io-static read performance measurement"])

Exception in thread Thread-16:
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "dtrace_cython/consumer.pyx", line 480, in dtrace.DTraceConsumerThread.run (dtrace_cython/consumer.c:6456)
  File "dtrace_cython/consumer.pyx", line 386, in dtrace.DTraceContinuousConsumer.go (dtrace_cython/consumer.c:5524)
  File "dtrace_cython/consumer.pyx", line 400, in dtrace.DTraceContinuousConsumer.go (dtrace_cython/consumer.c:5367)
Exception: ('Unable to compile the script: ', 'in action list: self->rstart has not yet been declared or assigned')



With predicate speed: KiBytes/sec
********************************************************************************
* Finished io-static read performance measurement
********************************************************************************


## Buffer Size Constant/Vary IO

In [ ]:
# Imports
import time

# Setup
BUF_START = 512
IO_SIZES = [512 * 2 ** exp for exp in range(4, 16)]
FILES = ["lab1/iofile", "/dev/zero"]
EXECUTABLES = ["io/io-static", "io/io-dynamic"]
BUFFER_SIZE = 512 * 2 ** 4 # Keep constant at default 16KB
RUNS = range(1, 16) # 15 runs

# The meat

outputs = {}
for exe in EXECUTABLES:
    print("\t ==> Running {}".format(exe))
    runs = []
    for filename in FILES:
        print("\t\t ==> Reading from {}".format(filename))
        for io_size in IO_SIZES:
            results = []
            for i in RUNS:
                out = !{exe} -v -r -b {str(BUFFER_SIZE)} -t {str(io_size)} {filename}
                parsed = parse(out)
                results.append(parsed)
                time.sleep(2) # Sleep for 2 seconds to prevent warmup effect

            runs.append(results)
            times = [float(item['time']) for item in results]
            speeds = [float(item['speed'].split(' ')[0]) for item in results] # Need to split by ' ' because output speed has units attached to it (see io.c)
            avg_time = sum(times) / len(RUNS)
            avg_speed = sum(speeds) / len(RUNS)
            buf_sz = int(results[0]['buffersize'])
            tot_sz = int(results[0]['totalsize'])
            print("\t\t\t{0:2} bytes {1:2} bytes ({2:.2f} KBytes/sec): {3:.6f}s".format(buf_sz, tot_sz, avg_speed, avg_time))
    outputs.update({exe:runs})

In [ ]:
## Buffer Cache Bypass

* -B flag

In [13]:
# Imports
import time

# Setup
BUF_START = 512
IO_SIZES = [512 * 2 ** exp for exp in range(4, 16)]
FILES = ["lab1/iofile", "/dev/zero"]
EXECUTABLES = ["io/io-static", "io/io-dynamic"]
BUFFER_SIZE = 512 * 2 ** 4 # Keep constant at default 16KB
RUNS = range(1, 16) # 15 runs

# The meat

outputs = {}
for exe in EXECUTABLES:
    print("\t ==> Running {}".format(exe))
    runs = []
    for filename in FILES:
        print("\t\t ==> Reading from {}".format(filename))
        for io_size in IO_SIZES:
            results = []
            for i in RUNS:
                out = !time -p {exe} -v -r -q -B -b {str(BUFFER_SIZE)} -t {str(io_size)} {filename}
                print(out)

	 ==> Running io/io-static
		 ==> Reading from lab1/iofile
['real 0.01', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['real 0.00', 'user 0.00', 'sys 0.00']
['rea